In [106]:
import warnings
warnings.filterwarnings('ignore')

In [107]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [108]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Read the CSV and Perform Basic Data Cleaning

In [109]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [110]:
# Load the data
file_path = ('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head(1)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N


In [111]:
df2=df.drop(['issue_d', 'out_prncp_inv', 'total_pymnt_inv','next_pymnt_d'], axis = 1)
#encode paymnet plan, initial_list_status, application type, hardship flag, debt_settlment_flag....
le = LabelEncoder()
df3 = df2.copy()
df3['pymnt_plan'] = le.fit_transform(df2['pymnt_plan']) 
df3['initial_list_status'] = le.fit_transform(df2['initial_list_status'])
df3['application_type']= le.fit_transform(df2['application_type'])
df3['hardship_flag']= le.fit_transform(df2['hardship_flag'])
df3['debt_settlement_flag']= le.fit_transform(df2['debt_settlement_flag'])
df3['home_ownership'] = le.fit_transform(df2['home_ownership'])
df3['verification_status'] = le.fit_transform(df2['verification_status'])


# Split the Data into Training and Testing

In [112]:
#get x & y
#load status is the Y, so drop it for the X dataset
X = df3.drop(columns="loan_status") 
y = df3['loan_status'].to_frame() 

In [113]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [114]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

In [115]:
### Balanced Random Forest Classifier

from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators = 100, random_state =1)



In [116]:
#fit model
rf_model = rf_model.fit(X_train, y_train)
#make predictions
predictions = rf_model.predict(X_test)


In [117]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8218541121766928

In [130]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index = ["actual 0", "1"], columns = ["Predicated 0 ", "Predicted 1"])
cm_df

,Predicated 0,Predicted 1
actual 0,62,39
1,3026,14078


In [119]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.77      0.23      1.00      0.35      0.48      0.21       101
   low_risk       1.00      1.00      0.23      1.00      0.48      0.25     17104

avg / total       0.99      1.00      0.23      0.99      0.48      0.24     17205



In [120]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
#sort 
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)



[(0.08504256720955392, 'total_pymnt'),
 (0.07778377861955971, 'total_rec_prncp'),
 (0.06492868386072663, 'last_pymnt_amnt'),
 (0.055430178760801585, 'total_rec_int'),
 (0.035550464461343, 'int_rate'),
 (0.01946953188527412, 'out_prncp'),
 (0.01941774227105081, 'bc_open_to_buy'),
 (0.0193406048744143, 'total_bc_limit'),
 (0.018753010918536258, 'annual_inc'),
 (0.017778815819186593, 'total_rev_hi_lim'),
 (0.017324074612356997, 'mo_sin_old_il_acct'),
 (0.01725477801349016, 'mths_since_recent_inq'),
 (0.017196545553865697, 'installment'),
 (0.017011344493404204, 'revol_bal'),
 (0.016833995952795024, 'dti'),
 (0.01648351325281806, 'max_bal_bc'),
 (0.016217566189621127, 'mo_sin_old_rev_tl_op'),
 (0.01619139056846755, 'total_bal_ex_mort'),
 (0.01554689862789483, 'il_util'),
 (0.01513365456543744, 'avg_cur_bal'),
 (0.015120525974111254, 'mths_since_rcnt_il'),
 (0.01472119732983651, 'total_il_high_credit_limit'),
 (0.014719497580377059, 'all_util'),
 (0.01461075266140914, 'tot_hi_cred_lim'),
 (

### Easy Ensemble AdaBoost Classifier

In [121]:
# Train the EasyEnsembleClassifier
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(n_estimators=100,learning_rate=1)
model = abc.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [122]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, y_pred)
acc_score

0.995059575704737

In [129]:
cm = confusion_matrix(y_test, y_pred)  #create a DF from cm
cm_df = pd.DataFrame(cm, index = ["Actual 0", "Actual 1 "], columns = ["Predicated 0", "Predicted 1"])
cm_df

,Predicated 0,Predicted 1
Actual 0,23,78
Actual 1,7,17097


In [124]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

   high_risk       0.02      0.61      0.04       101
    low_risk       1.00      0.82      0.90     17104

    accuracy                           0.82     17205
   macro avg       0.51      0.72      0.47     17205
weighted avg       0.99      0.82      0.90     17205



In [125]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
#sort 
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)


[(0.08504256720955392, 'total_pymnt'),
 (0.07778377861955971, 'total_rec_prncp'),
 (0.06492868386072663, 'last_pymnt_amnt'),
 (0.055430178760801585, 'total_rec_int'),
 (0.035550464461343, 'int_rate'),
 (0.01946953188527412, 'out_prncp'),
 (0.01941774227105081, 'bc_open_to_buy'),
 (0.0193406048744143, 'total_bc_limit'),
 (0.018753010918536258, 'annual_inc'),
 (0.017778815819186593, 'total_rev_hi_lim'),
 (0.017324074612356997, 'mo_sin_old_il_acct'),
 (0.01725477801349016, 'mths_since_recent_inq'),
 (0.017196545553865697, 'installment'),
 (0.017011344493404204, 'revol_bal'),
 (0.016833995952795024, 'dti'),
 (0.01648351325281806, 'max_bal_bc'),
 (0.016217566189621127, 'mo_sin_old_rev_tl_op'),
 (0.01619139056846755, 'total_bal_ex_mort'),
 (0.01554689862789483, 'il_util'),
 (0.01513365456543744, 'avg_cur_bal'),
 (0.015120525974111254, 'mths_since_rcnt_il'),
 (0.01472119732983651, 'total_il_high_credit_limit'),
 (0.014719497580377059, 'all_util'),
 (0.01461075266140914, 'tot_hi_cred_lim'),
 (